Подготовка базы данных

In [28]:
import pandas as pd
import threading
import sys

In [29]:
data = pd.read_csv('Raw_data.csv')

In [30]:
data.shape

(371583, 21)

In [31]:
data = data.query("businessman_flag == 0")
data = data.drop(['term','businessman_flag'], axis=1)

In [32]:
data = data.fillna({'start_date': -1, 'fact_close_date': -1, 'card_type_name':-1})

In [33]:
data = data.astype({'start_date': int, 'fact_close_date': int , 'purchase_sum':int ,'purchase_sum':int ,'contract_sum':int , 'current_balance_avg_sum':int , 'current_balance_sum':int, 'current_debit_turn_sum':int, 'current_credit_turn_sum':int})

In [34]:
data.shape

(371583, 19)

In [35]:
data = data.drop_duplicates() # Избавление от дубликатов записей

In [36]:
data.loc[data.gender == 'М', "gender"] = 'M'
data.loc[data.gender == 'Ж', "gender"] = 'F'


In [37]:
data

,client_id,gender,birth_date,create_date,nonresident_flag,city,contract_sum,product_category_name,card_id,card_type_name,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum,card_type
0,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F
1,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M
2,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F
3,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M
4,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371578,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M
371579,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M
371580,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M
371581,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F


Поиск валидных записей

In [38]:
listUniqueCardID = data.card_id.unique()
len(listUniqueCardID)

2

In [39]:
def getlistEqualPartsOfNumberParts(intNub, intParts):
  d, r = divmod(intNub, intParts)
  return [d + (1 if i < r else 0) for i in range(intParts)]

In [40]:
def th_get_ActualUserData(gl_DataFrame, listUniqueCardID, intIndex):
    local_UsersData = pd.DataFrame()
    for i in range(len(listUniqueCardID)):
        if float(i%100) == 0:
            print(i)
            
        strPDCall = f"card_id == '{listUniqueCardID[i]}'" # Запрос

        #f_Data = data.query(strPDCall)
        #print( listUniqueCardID[i] , '-', f_Data.shape)

        local_Data = data.query(strPDCall).drop_duplicates()
        #print( listUniqueCardID[i] , '---', local_Data.shape,'\n')

        if (local_Data.shape[0] != 1):
              
            maxDebet =  local_Data['current_debit_turn_sum'].max()
            maxCredet = local_Data['current_credit_turn_sum'].max()
            maxPurchaseCount = local_Data['purchase_count'].max()
            
            thisDB = local_Data.query(f"current_debit_turn_sum == {maxDebet} & current_credit_turn_sum == {maxCredet} & purchase_count == {maxPurchaseCount}")
            
        else:
            thisDB = local_Data
        
        #print(i, "-", thisDB.shape, "-", listUniqueCardID[i],'\n')

        local_UsersData = pd.concat([local_UsersData, thisDB]) 
           
        #print(local_UsersData.shape)

    local_UsersData.to_csv(f'New_CSV_{intIndex}.csv')

    gl_DataFrame = pd.concat([gl_DataFrame, local_UsersData]) 
    print(gl_DataFrame.shape)

    print(f'Thread {intIndex} is done !')
    sys.exit

In [41]:
listParts = getlistEqualPartsOfNumberParts(len(listUniqueCardID),int(len(listUniqueCardID)/500))
listThreads = []
intCount = 0
gl_ActualUserData = pd.DataFrame()

for i in range(len(listParts)):

    if listParts[i] == 0 :
        break
    print(f'Thread {i} from {intCount} to {intCount+listParts[i]} is start - {listParts[i]}')
    listThreads.append( threading.Thread( name=f'Thread_{i}', target=th_get_ActualUserData, args=(gl_ActualUserData, listUniqueCardID[ intCount: intCount+listParts[i] ],i , ) ))
    intCount += listParts[i]+1
    listThreads[i].start()

ZeroDivisionError: integer division or modulo by zero

In [ ]:
gl_ActualUserData = pd.DataFrame()

for i in range(len(listParts)):
    gl_ActualUserData = pd.concat([gl_ActualUserData, pd.read_csv(f'Source_f_2/New_CSV_{i}.csv').drop_duplicates()]) 
gl_ActualUserData.shape

100
100
100
100
100
100100100


100100
100
100

100
100
100100100


100
100100

100
100
100
100
100
100
100
100
100
100
100100

100
100
100
100
100
100
100
100
100
100
100
100
100
200200

200
200
200
200200
200

200
200
200
200
200200

200
200
200200
200

200
200
200200200


200
200200
200

200
200200

200200
200

200
200
200
200
200
200
200
200
200200

200
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300300

300300
300

300
300
300
300
300
300
300


(16313, 20)

In [ ]:
count = 0
for i in range(len(listParts)):
    count += pd.read_csv(f'Source_f_2/New_CSV_{i}.csv').shape[0]

400
400
400
400
400
400
400
400
400400

400
400
400400

400
400
400
400
400
400
400
400
400
400400

400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400400

400
(200, 19)
Thread 44 is done !
500
(223, 19)
Thread 28 is done !
500
(285, 19)
Thread 31 is done !
500
500
(147, 19)
Thread 16 is done !
500
500
(222, 19)
Thread 27 is done !
500
500500

(180, 19)(214, 19)500

Thread 5 is done !

Thread 26 is done !
(164, 19)
Thread 10 is done !
500
500
(188, 19)
Thread 0 is done !
(316, 19)
Thread 32 is done !
(182, 19)
Thread 3 is done !
500
500
500
500
(184, 19)(163, 19)
Thread 15 is done !

Thread 11 is done !
(206, 19)
Thread 38 is done !
500
500
(205, 19)
Thread 43 is done !
(350, 19)
Thread 33 is done !
500500

(210, 19)
Thread 36 is done !
500
500
(174, 19)
Thread 9 is done !
(191, 19)
Thread 20 is done !
500
500
500
(240, 19)
Thread 35 is done !
(181, 19)
Thread 1 is done !
500
500500
500
500

(146, 19)(196, 19)
Thread 18 is done !

Thread 8 is done !
(155, 19)
Thread

In [ ]:
gl_ActualUserData.drop(['Unnamed: 0'], axis=1)

(175, 19)
Thread 6 is done !
500


,client_id,gender,birth_date,create_date,nonresident_flag,city,contract_sum,product_category_name,card_id,card_type_name,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum,card_type
0,fe60b594364f9f636266ed1ef4f89c32,Ж,1985,2020-01-29,R,Гусь-Хрустальный,0,Договор на текущий счет для дебетовой карты,a83cbc33f164deae41b3339141923cc7,Visa Platinum Rewards,2019,2021,16600,71,39700,25700,220600,201000,dc
1,3012cabca5885ed53d348d6e57dab5de,М,1951,2009-09-24,R,Ишимбай,0,Договор на текущий счет для дебетовой карты,8dae17a830bb05d151034df618efbcad,VISA Classic,2019,-1,4900,2,38000,26800,56300,31300,dc
2,d871ef96820b6c7a1ada8e01a772724e,Ж,1982,2006-12-25,R,Юрга,0,Договор на текущий счет для дебетовой карты,429ca8c43cdc5f3d8c9efc9069ee3fa6,NaN,-1,-1,0,0,0,0,0,0,dc
3,0bacbc2abd314f9fac031d249e485a7b,М,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7f4adb9187b867ebea15503298437a3,Visa Rewards,2021,-1,47800,12,0,0,51500,51500,cc
4,0bacbc2abd314f9fac031d249e485a7b,М,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7d13882154e965dca7070eb8d626f77,Eurocard/MasterCard World,2018,2022,47800,12,0,0,51500,51500,cc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,9acb18399b830419bac9653b52754646,М,1986,2015-05-20,R,Уфа,0,Договор на текущий счет для дебетовой карты,d090cb20c22e2643ec366daf8d26d646,NaN,-1,-1,0,0,0,0,0,0,dc
326,8b3206ca8d77aa2446824971f47c531b,Ж,1996,2022-04-28,R,Казань,30000,Кредитная карта,5a0041efbd5832765caf8b9f861bffa3,NaN,-1,-1,22700,7,0,0,22700,22700,dc
327,d69d3201104ab393c0638839a7284ad7,М,1953,2018-08-08,R,Пятигорск,200000,Кредитная карта,a78c73e84e893179de17071583cf5084,NaN,-1,-1,0,0,0,300,9900,10100,dc
328,a936fb30e2268d2874cdcaa221032915,М,1997,2022-05-12,R,Кемерово,15000,Кредитная карта,e3304eaf55891a175b6abf8030b99da1,NaN,-1,-1,15000,33,0,0,15000,15000,dc


In [ ]:
gl_ActualUserData.to_csv('RTG_CSV.csv')

(472, 19)
Thread 34 is done !
